In [1]:
!pip install datasets

In [2]:
import time
import torch
import torch.nn as nn
import numpy as np
import random
from torch import optim
import matplotlib.pyplot as plt
from typing import List

from torch.utils.data import Dataset, DataLoader, RandomSampler
import tqdm
from scipy.stats import ttest_ind
# from bus_transformer import *
from datasets import load_dataset
from transformers import AutoTokenizer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
seq_len = 256
batch_size = 16
print(DEVICE)

cuda


In [3]:
!nvidia-smi

Mon Oct 28 11:24:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:2D:00.0  On |                  N/A |
|  0%   51C    P8              37W / 320W |    760MiB / 10240MiB |     10%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from graphviz import Digraph
from tqdm.notebook import trange, tqdm
from torch.autograd import Variable

# make_dot was moved to https://github.com/szagoruyko/pytorchviz
# from torchviz import make_dot

- limit sequences to 128
- limit tasks to sentence classification
- use single sequence training without NSP
-


In [5]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, d_internal):
        super().__init__()

        self.W_Q = torch.nn.Linear(d_model, d_internal, False)
        self.W_K = torch.nn.Linear(d_model, d_internal, False)
        self.W_V = torch.nn.Linear(d_model, d_internal, False)

        self.SoftMax = torch.nn.Softmax(dim=-1)


        self.d_model = d_model
        self.d_internal = d_internal
        self.norm = torch.tensor(d_model**-0.5)
        self.tril = torch.tril(torch.ones(seq_len, seq_len, device=DEVICE))

    def expand(self, d_mnew, d_inew):

        self.W_Q.weight.data = torch.cat([self.W_Q.weight.data, torch.zeros(d_inew - self.d_internal, self.d_model, device=DEVICE)], dim=0)
        self.W_Q.weight.data = torch.cat([self.W_Q.weight.data, torch.zeros(d_inew, d_mnew - self.d_model, device=DEVICE)], dim=1)
        for i in range(self.d_internal, d_inew):
            self.W_Q.weight.data[i][i] = self.W_Q.weight.data[i][i] if self.W_Q.weight.data[i][i] != 0 else 1

        self.W_K.weight.data = torch.cat([self.W_K.weight.data, torch.zeros(d_inew - self.d_internal, self.d_model, device=DEVICE)], dim=0)
        self.W_K.weight.data = torch.cat([self.W_K.weight.data, torch.zeros(d_inew, d_mnew - self.d_model, device=DEVICE)], dim=1)
        for i in range(self.d_internal, d_inew):
            self.W_K.weight.data[i][i] = self.W_K.weight.data[i][i] if self.W_K.weight.data[i][i] != 0 else 1

        self.W_V.weight.data = torch.cat([self.W_V.weight.data, torch.zeros(d_inew - self.d_internal, self.d_model, device=DEVICE)], dim=0)
        self.W_V.weight.data = torch.cat([self.W_V.weight.data, torch.zeros(d_inew, d_mnew - self.d_model, device=DEVICE)], dim=1)
        for i in range(self.d_internal, d_inew):
            self.W_V.weight.data[i][i] = self.W_V.weight.data[i][i] if self.W_V.weight.data[i][i] != 0 else 1

        self.d_internal = d_inew
        self.d_model = d_mnew
        self.SoftMax = torch.nn.Softmax(dim=-1)
        self.tril = torch.tril(torch.ones(seq_len, seq_len, device=DEVICE))



    def forward(self, input_vecs):
        B, T, C = input_vecs.shape

        Q = self.W_Q(input_vecs)
        K = self.W_K(input_vecs)
        V = self.W_V(input_vecs)

        weights = Q @ K.transpose(-2, -1) * C**-0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        Attn = self.SoftMax(weights)


        out = Attn @ V

        return out

In [6]:
class Transformer(nn.Module):
    def __init__(self, d_model, vocab_size, num_heads, d_hidden):
        super().__init__()
        self.d_model = d_model
        self.d_internal = d_model//num_heads
        self.num_heads = num_heads
        self.vocab_size = vocab_size
        self.d_hidden = d_hidden

        self.heads = nn.ModuleList([AttentionHead(d_model, self.d_internal) for _ in range(num_heads)])
        self.Softmax = torch.nn.LogSoftmax(dim=-1)
        self.FFN = torch.nn.Sequential(
            torch.nn.Linear(self.d_model, self.d_hidden),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.d_hidden, self.d_model),
        )
        self.W_O = torch.nn.Linear(d_model, d_model, False)
        self.layernorm = torch.nn.LayerNorm(d_model)



    def forward(self, x):
        """
        :param x: input embeddings
        :return: output of decoder block, same shape as input
        """
        t = x
        t = torch.cat([head(t) for head in self.heads], dim=-1)
        t = self.W_O(t)
        t1 = self.layernorm(t + x)
        # t = self.relu(self.cout(self.FFN(self.connection(t1))))
        t = self.FFN(t1)
        t = self.layernorm(t + t1)

        return t



    def expand(self, d_mnew, d_inew):

        # self.connection = torch.nn.Linear(d_mnew, self.d_hidden)
        # self.cout = torch.nn.Linear(self.d_hidden, d_mnew)

        self.FFN = torch.nn.Sequential(
            torch.nn.Linear(d_mnew, self.d_hidden),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(self.d_hidden, d_mnew),
        )
        self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew-self.d_model, self.d_model, device=DEVICE)], dim=0)
        self.W_O.weight.data = torch.cat([self.W_O.weight.data, torch.zeros(d_mnew, d_mnew-self.d_model,  device=DEVICE)], dim=1)
        self.layernorm = torch.nn.LayerNorm(d_mnew)
        for i in range(self.d_model+1, d_mnew):
            self.W_O.weight.data[i][i] = 1

        for head in self.heads:
            head.expand(d_mnew, d_inew)

        self.Softmax = torch.nn.LogSoftmax(dim=-1)
        self.d_model = d_mnew
        self.d_internal = d_inew
        self.to(DEVICE)




In [7]:
class Decoder(nn.Module):
    def __init__(self, num_blocks, d_model, d_hidden, vocab_size, num_heads):
        super().__init__()
        self.num_blocks = num_blocks
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.num_heads = num_heads
        self.SoftMax = torch.nn.LogSoftmax(dim=-1)
        self.blocks = torch.nn.ModuleList([Transformer(d_model, vocab_size, num_heads, d_hidden) for _ in range(num_blocks)])
        self.d_hidden = d_hidden

        # self.connection = torch.nn.Linear(d_model, d_hidden)
        self.FFN = torch.nn.Sequential(
            torch.nn.Linear(d_model, vocab_size),
            torch.nn.LogSoftmax(dim=-1),
        )
        self.dout = torch.nn.Dropout(0.1)

        self.embeddings = torch.nn.Embedding(vocab_size, d_model, device=DEVICE)
        self.pos_embedding = None
        # self.pos_embedding = torch.nn.Embedding(seq_len, d_model, device=DEVICE)
        self.generate_pos_embed(d_model)


    def forward(self, x):
        x = self.embeddings(x) + self.pos_embedding(torch.arange(x.shape[-1], device=DEVICE))
        x = self.dout(x)
        t = x
        for head in self.blocks:
            t = head(t) + t

        ret = self.FFN(t)

        return ret

    def generate_pos_embed(self, d_model):
        # TODO: make more efficient
        pos_em = torch.zeros((seq_len, d_model))
        for pos in range(seq_len):
            for i in range(d_model):
                if i % 2 == 0:
                    pos_em[pos][i] += torch.sin(torch.tensor(pos/(10000**(2*i/d_model))))
                else:
                    pos_em[pos][i] += torch.cos(torch.tensor(pos/(10000** (2*i/d_model))))

        self.pos_embedding = torch.nn.Embedding.from_pretrained(pos_em, freeze=True)




    def expand(self, d_mnew):
        d_inew = d_mnew // self.num_heads
        self.FFN = torch.nn.Sequential(
            torch.nn.Linear(d_mnew, self.vocab_size),
            torch.nn.LogSoftmax(dim=-1),
        )

        self.layernorm = torch.nn.LayerNorm(d_mnew, device=DEVICE)
        for block in self.blocks:
            block.expand(d_mnew, d_inew)

        self.embeddings = torch.nn.Embedding.from_pretrained(torch.cat([self.embeddings.weight, torch.zeros(self.vocab_size, d_mnew-self.d_model, device=DEVICE).uniform_()], dim=1))
        # self.pos_embedding = torch.nn.Embedding.from_pretrained(torch.cat([self.pos_embedding.weight, torch.zeros(seq_len, d_mnew-self.d_model, device=DEVICE).uniform_()], dim=1))
        self.generate_pos_embed(d_mnew)
        # self.embeddings = torch.nn.Embedding(self.vocab_size, d_mnew)
        # self.pos_embedding = torch.nn.Embedding(seq_len, d_mnew)

        self.d_model = d_mnew
        self.d_internal = d_inew
        self.to(DEVICE)

In [8]:
model = Decoder(num_blocks=12, d_model=768, d_hidden=768*4, vocab_size=50257, num_heads=12)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

162.440785 M parameters


In [14]:
data = load_dataset('Salesforce/wikitext', 'wikitext-103-raw-v1')
# data = load_dataset('tiny_shakespeare')
train = data['train']
validation = data['validation']
test = data['test']

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [15]:
train.column_names

['text']

In [16]:
train['text'][0:100]

['',
 ' = Valkyria Chronicles III = \n',
 '',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more f

In [18]:
train_join = ' '.join(train['text'])

In [19]:
train_join[0:100]

'  = Valkyria Chronicles III = \n   Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリ'

In [11]:
chars = sorted(set(next(iter(train['text']))))
len(chars)


0

In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
train = data['train']['text'][0]
validation = data['validation']['text'][0]
test = data['test']['text'][0]

In [ ]:
test

"rance ta'en\nAs shall with either part's agreement stand?\n\nBAPTISTA:\nNot in my house, Lucentio; for, you know,\nPitchers have ears, and I have many servants:\nBesides, old Gremio is hearkening still;\nAnd happily we might be interrupted.\n\nTRANIO:\nThen at my lodging, an it like you:\nThere doth my father lie; and there, this night,\nWe'll pass the business privately and well.\nSend for your daughter by your servant here:\nMy boy shall fetch the scrivener presently.\nThe worst is this, that, at so slender warning,\nYou are like to have a thin and slender pittance.\n\nBAPTISTA:\nIt likes me well. Biondello, hie you home,\nAnd bid Bianca make her ready straight;\nAnd, if you will, tell what hath happened,\nLucentio's father is arrived in Padua,\nAnd how she's like to be Lucentio's wife.\n\nBIONDELLO:\nI pray the gods she may with all my heart!\n\nTRANIO:\nDally not with the gods, but get thee gone.\nSignior Baptista, shall I lead the way?\nWelcome! one mess is like to be your cheer:

In [ ]:
train_data = encode(train)
train_data[:10]

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]

In [ ]:
val_data = encode(validation)
test_data = encode(test)

In [ ]:
def batch(s):
    data = train_data if s == 'train' else val_data
    ix = torch.randint(len(data) - seq_len, (batch_size,))
    x = torch.stack([torch.tensor(data[i:i+seq_len], device=DEVICE) for i in ix])
    y = torch.stack([torch.tensor(data[i+1:i+seq_len+1], device=DEVICE) for i in ix])
    return x, y

In [ ]:
xb, yb = batch('train')
xb

tensor([[19,  1, 30,  ..., 30, 21, 15],
        [13, 30, 17,  ..., 56,  1, 42],
        [43, 56, 43,  ..., 10,  0, 28],
        ...,
        [45,  1, 39,  ..., 47, 43, 60],
        [42, 39, 51,  ..., 52, 43,  1],
        [61, 47, 58,  ..., 35, 46, 47]], device='cuda:0')

In [ ]:
yb

tensor([[ 1, 30, 21,  ..., 21, 15, 20],
        [30, 17, 32,  ...,  1, 42, 39],
        [56, 43, 47,  ...,  0, 28, 50],
        ...,
        [ 1, 39, 57,  ..., 43, 60, 43],
        [39, 51, 52,  ..., 43,  1, 59],
        [47, 58, 46,  ..., 46, 47, 41]], device='cuda:0')

In [ ]:
len(data['test']['text'][0])

55770

In [ ]:
max_iters = 5000
eval_interval = 1000
eval_iters = 200
test_iters = 1000

In [ ]:
@torch.no_grad()
def estimate_loss(s=['train', 'val']):
    out = {}
    model.eval()
    for split in s:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = batch(split)
            logits= model(X)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = torch.nn.functional.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
def acc(model):
    model.eval()
    a = []

    for k in range(test_iters):
        X, Y = batch("test")
        logits = torch.argmax(model(X), dim=-1)
        x = [[0. if logits[sample][i] != Y[sample][i] else 1. for i in range(seq_len)] for sample in range(batch_size)]
        a.append(x)
    return torch.mean(torch.tensor(a))




In [ ]:
def train(model, lr=1e-3, min_lr=1e-6, max_it=max_iters):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iters, min_lr)
    for iter in tqdm(range(max_it)):

        # every once in a while evaluate the loss on train and val sets
        # if iter % eval_interval == 0 or iter == max_iters - 1:
        #     losses = estimate_loss()
        #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, lr {optimizer.param_groups[0]['lr']}")

        # sample a batch of data
        xb, yb = batch('train')

        # evaluate the loss
        logits = model(xb)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = yb.view(B*T)
        loss = torch.nn.functional.cross_entropy(logits, targets)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        scheduler.step()


In [ ]:
model = Decoder(num_blocks=4, d_model=128, vocab_size=len(chars), num_heads=4, d_hidden=64*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

0.559937 M parameters


In [ ]:
train(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
def eval(model):
    losses = estimate_loss(['train', 'val', 'test'])

    print(f"step {iter}:\t train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, test loss {losses['test']:.4f}")

In [ ]:
eval(model)

step <built-in function iter>:	 train loss 1.6151, val loss 1.7570, test loss 1.7607


In [ ]:
def train_transfer(model, transfer_step=900, target_size=1024, lr=1e-3, min_lr=1e-6):
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iters, min_lr)
    for iter in tqdm(range(1, max_iters)):

        # every once in a while evaluate the loss on train and val sets
        # if iter % eval_interval == 0 or iter == max_iters - 1 or iter == 1:
        #     losses = estimate_loss()
        #     print(f"step {iter}:\t train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, lr {optimizer.param_groups[0]['lr']}")

        if iter == transfer_step:
        # if iter <= 1000 and iter % 500 == 0:
            eval(model)
            model.expand(target_size)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 100, 0.5)
            print('at step {}: expanded model to: {} M parameters'.format(iter, sum(p.numel() for p in model.parameters())/1e6))
            model.to('cpu')
            model.to(DEVICE)    # Shortcut to recompile gradient backprop since the model changed sizes
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, max_iters-transfer_step, min_lr)
            loss_func = torch.nn.CrossEntropyLoss()
        # sample a batch of data
        xb, yb = batch('train')

        # evaluate the loss
        logits = model(xb)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = yb.view(B*T)
        loss = loss_func(logits, targets)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [ ]:
model = Decoder(num_blocks=4, d_model=64, vocab_size=len(chars), num_heads=4, d_hidden=64*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

0.214977 M parameters


In [ ]:
def train_transfer_gradual(model, transfer_step=600, final_size=128, start_size=64, final_bus_step=1200,  lr=1e-3, min_lr=1e-5):
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, min_lr)
    step = final_bus_step // transfer_step
    step_size = (final_size-start_size)//step
    for iter in tqdm(range(1, max_iters)):

        # every once in a while evaluate the loss on train and val sets
        # if iter % eval_interval == 0 or iter == max_iters - 1 or iter == 1:
        #     losses = estimate_loss()
        #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # if iter <= 1000 and iter % 500 == 0:
        if iter % transfer_step == 0 and iter <= final_bus_step:
            start_size += step_size
            model.expand(start_size)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 100, 0.5)
            print('at step {}: expanded model to: {} M parameters\tmodel_size: {}'.format(iter, sum(p.numel() for p in model.parameters())/1e6, start_size))
            model.to('cpu')
            model.to(DEVICE)    # Shortcut to recompile gradient backprop since the model changed sizes

            loss_func = torch.nn.CrossEntropyLoss()
        # sample a batch of data
        xb, yb = batch('train')

        # evaluate the loss
        logits = model(xb)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = yb.view(B*T)
        loss = loss_func(logits, targets)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        scheduler.step()

# WORK BENCH

### Proof of concept


In [ ]:
model = Decoder(num_blocks=4, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=800, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

8.762689 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

at step 800: expanded model to: 12.730433 M parameters
step <built-in function iter>:	 train loss 1.3517, val loss 1.5096, test loss 1.5172


In [ ]:

model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4)
eval(model)

12.729409 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3611, val loss 1.5403, test loss 1.5416


In [ ]:
model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4, max_it=4200)
eval(model)

12.729409 M parameters


  0%|          | 0/4200 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3972, val loss 1.5633, test loss 1.5603


In [ ]:
model = Decoder(num_blocks=12, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=800, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

26.089793 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

at step 800: expanded model to: 37.924929 M parameters
step <built-in function iter>:	 train loss 1.3910, val loss 1.5503, test loss 1.5542


In [ ]:

model = Decoder(num_blocks=12, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4)
eval(model)

37.923905 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3977, val loss 1.5536, test loss 1.5493


In [ ]:
model = Decoder(num_blocks=4, d_model=512, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train(model, lr=1e-3, min_lr=1e-4, max_it=5000)
eval(model)

12.729409 M parameters


  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3588, val loss 1.5308, test loss 1.5267


In [ ]:
train(model, lr=5e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.2162, val loss 1.4678, test loss 1.4707


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1403, val loss 1.4538, test loss 1.4568


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.0699, val loss 1.4872, test loss 1.4800


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9942, val loss 1.5202, test loss 1.5071


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9154, val loss 1.5392, test loss 1.5573


In [ ]:
train(model, lr=5e-4, min_lr=1e-5, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.8550, val loss 1.5745, test loss 1.5831


long training with a transfer first

In [ ]:
model = Decoder(num_blocks=4, d_model=384, vocab_size=len(chars), num_heads=8, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")
train_transfer(model, transfer_step=4500, target_size=512, lr=1e-3, min_lr=1e-4)
eval(model)

8.762689 M parameters


  0%|          | 0/4999 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.3308, val loss 1.4914, test loss 1.4965
at step 4500: expanded model to: 12.730433 M parameters
step <built-in function iter>:	 train loss 1.7626, val loss 1.8929, test loss 1.8901


In [ ]:
train(model, lr=5e-4, min_lr=1e-4)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.2705, val loss 1.4714, test loss 1.4686


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1942, val loss 1.4544, test loss 1.4493


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.1257, val loss 1.4656, test loss 1.4595


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 1.0627, val loss 1.4846, test loss 1.4792


In [ ]:
train(model, lr=1e-4, min_lr=1e-4, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9944, val loss 1.5142, test loss 1.5011


In [ ]:
train(model, lr=1e-4, min_lr=5e-5, max_it=5000)
eval(model)

  0%|          | 0/5000 [00:00<?, ?it/s]

step <built-in function iter>:	 train loss 0.9002, val loss 1.5354, test loss 1.5442


In [ ]:
# GPT-3-small model params ~125M params
model = Decoder(num_blocks=12, d_model=768, vocab_size=50257, num_heads=12, d_hidden=512*4)
model.to(DEVICE)
print(sum(p.numel() for p in model.parameters())/1e6, "M parameters")

143.455825 M parameters


In [ ]:
model

Decoder(
  (SoftMax): LogSoftmax(dim=-1)
  (blocks): ModuleList(
    (0-3): 4 x Transformer(
      (heads): ModuleList(
        (0-7): 8 x AttentionHead(
          (W_Q): Linear(in_features=512, out_features=64, bias=False)
          (W_K): Linear(in_features=512, out_features=64, bias=False)
          (W_V): Linear(in_features=512, out_features=64, bias=False)
          (SoftMax): Softmax(dim=-1)
        )
      )
      (Softmax): LogSoftmax(dim=-1)
      (FFN): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.1, inplace=False)
        (2): Linear(in_features=1024, out_features=1024, bias=True)
        (3): ReLU()
        (4): Dropout(p=0.1, inplace=False)
        (5): Linear(in_features=1024, out_features=1024, bias=True)
        (6): ReLU()
        (7): Dropout(p=0.1, inplace=False)
      )
      (W_O): Linear(in_features=512, out_features=512, bias=False)
      (layernorm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((512,), eps=1e-0